In [27]:
from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime

## Group Assignment
### Team Number: 16
### Team Member Names: Matthew Gong, Arya Patel, Saihej Singh
### Team Strategy Chosen: __________(RISKY OR SAFE)

In [31]:
possibleTickers = pd.read_csv("Tickers_Example.csv")
tickers = []
tickers_hist = {}
tickers_long_hist = {}
minVolume = 150000
allowed_currencies = ["USD", "CAD"]
start_date = "2023-01-01"
end_date = "2023-10-31"
long_start_date = "2018-10-31"
threshold_day = 18
days_in_month = []

for ticker in possibleTickers[possibleTickers.columns.values[0]]:
    currTicker = yf.Ticker(ticker)
    currHist = currTicker.history(start=start_date, end=end_date)
    currFastInfo = currTicker.fast_info

    if currHist.empty:
        print(ticker, "is delisted.")
    elif ticker in tickers:
        print(ticker, "is a duplicate.")
    else:
        tickers_hist[ticker] = currHist
        tickers_long_hist[ticker] = currTicker.history(start=long_start_date, end=end_date)

        group_months = currHist.reset_index().groupby(pd.Grouper(key='Date', freq='M'))
        dfs = [group for _, group in group_months]
        filtered_dfs = []
        for i, df in enumerate(dfs):
            #recombine all the months with more trading days than set threshold
            if len(df.index) >= threshold_day:
                days_in_month.append(len(df.index))
                filtered_dfs.append(df)
        currHist_updated = pd.concat(filtered_dfs)
        currHist_updated.set_index("Date", inplace=True)

        if currFastInfo["currency"] not in allowed_currencies:
            print(ticker, "is not in USD or CAD")
        elif currHist["Volume"].mean() * (np.mean(days_in_month)) < minVolume:
            print(ticker, "only has a volume of", currFastInfo["lastVolume"], "and average days of", np.mean(days_in_month))
        else:
            tickers.append(ticker)

print(tickers)

AGN: No timezone found, symbol may be delisted


AGN is delisted.


CELG: No timezone found, symbol may be delisted


CELG is delisted.


MON: No timezone found, symbol may be delisted


MON is delisted.


RTN: No timezone found, symbol may be delisted


RTN is delisted.
YCBD is a duplicate.
['ABBV', 'ABT', 'ACN', 'AIG', 'AMZN', 'AXP', 'BA', 'BAC', 'BIIB', 'BK', 'BLK', 'BMY', 'C', 'CAT', 'CL', 'KO', 'LLY', 'LMT', 'MO', 'MRK', 'PEP', 'PFE', 'PG', 'PM', 'PYPL', 'QCOM', 'RY.TO', 'SHOP.TO', 'T.TO', 'TD.TO', 'TXN', 'UNH', 'UNP', 'UPS', 'USB', 'YCBD']


In [32]:
marketTicker = yf.Ticker("^GSPC")
marketHist = marketTicker.history(start=long_start_date, end=end_date)

beta = {}

def beta_volatility(ticker):
    prices = pd.DataFrame(tickers_long_hist[ticker]["Close"])
    prices.columns = [ticker]
    prices["Market"] = marketHist['Close']

    monthly_change = prices.resample("M").ffill().pct_change()
    monthly_change = monthly_change.drop(index=monthly_change.index[0])

    return (monthly_change.cov()/monthly_change["Market"].var()).iat[0, 1]


for ticker in tickers:
    beta[ticker] = beta_volatility(ticker)

sorted(beta.items(), key=lambda x:x[1], reverse=True)


C:\Users\matth\AppData\Local\Temp\ipykernel_180\48051138.py:11: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Call ffill before calling pct_change to retain current behavior and silence this warning.
  monthly_change = prices.resample("M").ffill().pct_change()
C:\Users\matth\AppData\Local\Temp\ipykernel_180\48051138.py:11: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Call ffill before calling pct_change to retain current behavior and silence this warning.
  monthly_change = prices.resample("M").ffill().pct_change()
C:\Users\matth\AppData\Local\Temp\ipykernel_180\48051138.py:11: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Call ffill before calling pct_change to retain current behavior and silence this warning.
  monthly_change = prices.resample("M").ffil

[('YCBD', 2.1401685163092337),
 ('SHOP.TO', 1.8105683661552798),
 ('C', 1.5566632037877992),
 ('BA', 1.4762120958789395),
 ('BAC', 1.3851004089369816),
 ('PYPL', 1.379281690066955),
 ('BLK', 1.2835913572078048),
 ('ACN', 1.2304498749633876),
 ('QCOM', 1.2167443165555458),
 ('AXP', 1.214661990624904),
 ('AMZN', 1.1719942897819302),
 ('UPS', 1.104947635301972),
 ('UNP', 1.102517118852132),
 ('BK', 1.095338473906119),
 ('CAT', 1.087943298515294),
 ('AIG', 1.035101793224145),
 ('USB', 1.0267306045263955),
 ('TXN', 1.016827553644017),
 ('PM', 0.7766934886658905),
 ('ABT', 0.6933942169732435),
 ('MO', 0.6535227838115535),
 ('TD.TO', 0.6386753994827417),
 ('UNH', 0.6321135065675938),
 ('PFE', 0.6305321953920194),
 ('LMT', 0.5987255032149958),
 ('KO', 0.5908110704103807),
 ('PEP', 0.5726421817967551),
 ('RY.TO', 0.5664454522261506),
 ('ABBV', 0.5070224586012692),
 ('PG', 0.47479098903495087),
 ('CL', 0.4606673166245231),
 ('T.TO', 0.45967015160937147),
 ('BMY', 0.3853920009847),
 ('MRK', 0.364

In [30]:
def correlation(stock1, stock2):
    stock1Hist = tickers_hist[stock1].copy()
    stock2Hist = tickers_hist[stock2].copy()

    prices = pd.DataFrame(stock1Hist["Close"])
    prices.columns = [stock1]
    prices[stock2] = stock2Hist["Close"]
    
    monthly_returns = prices.resample('M').ffill().pct_change()
    monthly_returns.drop(index=monthly_returns.index[0], inplace=True)

    return monthly_returns.corr()

## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

Insert Names Here.